In [297]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day16input.txt")

# --- Day 16: Proboscidea Volcanium ---

The sensors have led you to the origin of the distress signal: yet another handheld device, just like the one the Elves gave you. However, you don't see any Elves around; instead, the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.

The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!

You need to get the elephants out of here, quickly. Your device estimates that you have 30 minutes before the volcano erupts, so you don't have time to go back out the way you came in.

You scan the cave for other options and discover a network of pipes and pressure-release valves. You aren't sure how such a system got into a volcano, but you don't have time to complain; your device produces a report (your puzzle input) of each valve's flow rate if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.

There's even a valve in the room you and the elephants are currently standing in labeled AA. You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?

In [298]:
import networkx as nx
import operator as op

max_time = 30
start_point = "AA"



For example, suppose you had the following scan output:

    

In [299]:
test_data = ["Valve AA has flow rate=0; tunnels lead to valves DD, II, BB",
"Valve BB has flow rate=13; tunnels lead to valves CC, AA",
"Valve CC has flow rate=2; tunnels lead to valves DD, BB",
"Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE",
"Valve EE has flow rate=3; tunnels lead to valves FF, DD",
"Valve FF has flow rate=0; tunnels lead to valves EE, GG",
"Valve GG has flow rate=0; tunnels lead to valves FF, HH",
"Valve HH has flow rate=22; tunnel leads to valve GG",
"Valve II has flow rate=0; tunnels lead to valves AA, JJ",
"Valve JJ has flow rate=21; tunnel leads to valve II"]

order: Ax, D20, Cx, B13, Ax, Ix, J21, Ix, Ax, D20, Ex, Fx, Gx, H22, Gx, Fx, E3, Dx, C2, 

In [300]:
def view_graph(V):
    for n in nx.nodes(V):
        print(n, nx.get_node_attributes(V, "flowrate")[n], nx.edges(V, n))

#print(list(nx.get_node_attributes(V_test, "flowrate").items()))

In [301]:
class Elf_Volcano:

    def __init__(self, edges, labels):
        self.G = nx.Graph()
        #V.add_nodes_from(labels) # actually not needed, any unconnected nodes become unnecessary (unless start node...)
        self.G.add_edges_from(edges)
        nx.set_node_attributes(self.G, labels, "flowrate")
        self.priorityQ = self.gen_priorityQ_list_by_flowrate()
        #print(priorityQ)
        self.accummulated_pressure = 0
        self.sum_open_pressure = 0
        self.current_valve = None
        self.target = None
        self.time_left = -1
        self.current_path = None
        self.open_valves = []
    

    def gen_priorityQ_list_by_flowrate(self):
        v_list = list(nx.get_node_attributes(self.G, "flowrate").items())
        v_list.sort(reverse=True, key = op.itemgetter(1))
        return_list = []
        for v in v_list:
            if (v[1]>0):
                return_list.append(v)
        return return_list

    def maximise_pressure(self, start_point, time_left):
        self.current_valve = start_point
        self.time_left = time_left
        self.open_valves = []
        self.set_target()
        # print("== Minute ", time_left-self.time_left+1, " == \n open valves: ", self.open_valves," releasing ",self.sum_open_pressure, "\nYou move to ", self.current_valve)
        while self.time_left>0:
            self.take_turn(time_left)
        return self.accummulated_pressure


    def take_turn(self, time_left):
        if self.time_left>0:
            self.open_valve(time_left)
        if self.time_left>0:
            self.choose_tunnel_based_on_global_best(time_left)

    def time_tick(self, time_left, msg):
        self.time_left = self.time_left - 1
        self.accummulated_pressure = self.accummulated_pressure + self.sum_open_pressure
        print("== Minute "+ str(time_left-self.time_left)+  " == \n open valves: "+ str(self.open_valves)+" releasing "+str(self.sum_open_pressure)+ msg +"\n")

    def open_valve(self, time_left): 
        flow = nx.get_node_attributes(self.G,"flowrate")[self.current_valve]
        if self.target is not None:
            if flow>0 and self.current_valve not in self.open_valves and self.current_valve == self.target[0]:
                msg = "\nYou open valve "+ self.current_valve
                self.time_tick(time_left, msg)
                self.priorityQ.remove(self.target)
                self.open_valves.append(self.target)
                self.sum_open_pressure = self.sum_open_pressure + self.target[1]
                if len(self.priorityQ) >0:
                    self.set_target()
                else:
                    self.target = None


    def choose_tunnel_based_on_global_best(self, time_left):
        # print(self.current_path)
        if self.target is not None:
            for n in range(0, len(self.current_path)-1):
                self.current_valve = self.current_path[n+1] #we know there's an edge
                msg = "\nYou move to "+ str(self.current_valve)
        else: 
            msg = ""
        self.time_tick(time_left, msg)


    def set_target(self):
        choices = []
        for c in self.priorityQ:
            spl = nx.shortest_path_length(self.G, self.current_valve, c[0])
            time = (self.time_left-spl)*c[1]*-1
            #print(spl, time, c[0])
            choices.append((spl, time, c))
        choices.sort()
        nearest = choices[0][0]
        selected = [c for c in choices if c[0]==nearest]
        selected.sort(key=op.itemgetter(1))
        #print(selected)
        self.target = selected[0][2]
        self.current_path = nx.shortest_path(self.G, self.current_valve, self.target[0])
        

    # def choose_tunnel_based_on_local_best(self):
    #     possibles = list(self.G.edges(self.current_valve))
    #     next_step = (None, -1)
    #     #fr = -1
    #     for p in possibles:
    #         flow = nx.get_node_attributes(self.G,"flowrate")[p[1]]
    #         open = nx.get_node_attributes(self.G,"open")[p[1]]
    #         # print(flow)
    #         # print(next_step)
    #         if open == False and flow>next_step[1]:
    #             next_step = (p[1], flow)
    #     self.current_valve = next_step[0]
    #     self.time_left = self.time_left - 1

    # # def open_valve_based_on_current_path(self): #volcano, chosen_valve, target, current_path, priorityQ, time_left, pressure):
    # #     #for now, assume only open if at top of Q
    # #     if self.target[0] == self.current_valve:
    # #         print("Opening valve ", self.current_valve)
    # #         self.time_left = self.time_left - 1
    # #         self.pressure = self.pressure + (self.target[1]*self.time_left)
    # #         self.target = self.priorityQ.pop(0)
    # #         self.current_path = nx.shortest_path(self.G, self.current_valve, self.target[0])


    # # def choose_tunnel_based_on_path(self):
    # #     possibles = list(self.G.edges(self.current_valve))
    # #     next_step = self.current_path[self.current_path.index(self.current_valve)+1]
    # #     #print(possibles)
    # #     if (self.current_valve, next_step) in possibles:
    # #         self.current_valve = next_step
    # #     else: 
    # #         print("problem with path")
    # #         self.current_valve = None
    # #     self.time_left = self.time_left - 1


In [302]:
def parse_data(data):
    labels = {}
    edges = []
    for d in data:
        words = d.split()
        #identify edges
        for e in words[9:]:
            edges.append((words[1], e[0:2]))
        #identify flow rates for valves
        fr = words[4].split("=")
        fr[1] = fr[1].removesuffix(";")
        labels[words[1]] = eval(fr[1])
    V = Elf_Volcano(edges, labels)
    return V


In [303]:
V_test = parse_data(test_data)
V_elf = parse_data(elf_data)
#V_test.gen_priorityQ_list_by_flowrate()
print(V_test.maximise_pressure(start_point, max_time))
print(V_elf.maximise_pressure(start_point, max_time))

== Minute 1 == 
 open valves: [] releasing 0
You move to DD

== Minute 2 == 
 open valves: [] releasing 0
You open valve DD

== Minute 3 == 
 open valves: [('DD', 20)] releasing 20
You move to EE

== Minute 4 == 
 open valves: [('DD', 20)] releasing 20
You open valve EE

== Minute 5 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You move to CC

== Minute 6 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You open valve CC

== Minute 7 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You move to BB

== Minute 8 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You open valve BB

== Minute 9 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You move to JJ

== Minute 10 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You open valve JJ

== Minute 11 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13), ('JJ', 21)] releasing 59
You move to HH

== Minute 12 == 
 open valves: [(

*2454 is too high*

In [304]:
V_test = parse_data(test_data)
# V_elf = parse_data(elf_data)
#V_test.gen_priorityQ_list_by_flowrate()
print(V_test.maximise_pressure(start_point, max_time))

== Minute 1 == 
 open valves: [] releasing 0
You move to DD

== Minute 2 == 
 open valves: [] releasing 0
You open valve DD

== Minute 3 == 
 open valves: [('DD', 20)] releasing 20
You move to EE

== Minute 4 == 
 open valves: [('DD', 20)] releasing 20
You open valve EE

== Minute 5 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You move to CC

== Minute 6 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You open valve CC

== Minute 7 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You move to BB

== Minute 8 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You open valve BB

== Minute 9 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You move to JJ

== Minute 10 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You open valve JJ

== Minute 11 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13), ('JJ', 21)] releasing 59
You move to HH

== Minute 12 == 
 open valves: [(

In [305]:
V_test = parse_data(test_data)
# V_elf = parse_data(elf_data)
#V_test.gen_priorityQ_list_by_flowrate()
print(V_test.maximise_pressure(start_point, max_time))

== Minute 1 == 
 open valves: [] releasing 0
You move to DD

== Minute 2 == 
 open valves: [] releasing 0
You open valve DD

== Minute 3 == 
 open valves: [('DD', 20)] releasing 20
You move to EE

== Minute 4 == 
 open valves: [('DD', 20)] releasing 20
You open valve EE

== Minute 5 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You move to CC

== Minute 6 == 
 open valves: [('DD', 20), ('EE', 3)] releasing 23
You open valve CC

== Minute 7 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You move to BB

== Minute 8 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2)] releasing 25
You open valve BB

== Minute 9 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You move to JJ

== Minute 10 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13)] releasing 38
You open valve JJ

== Minute 11 == 
 open valves: [('DD', 20), ('EE', 3), ('CC', 2), ('BB', 13), ('JJ', 21)] releasing 59
You move to HH

== Minute 12 == 
 open valves: [(


All of the valves begin closed. You start at valve AA, but it must be damaged or jammed or something: its flow rate is 0, so there's no point in opening it. However, you could spend one minute moving to valve BB and another minute opening it; doing so would release pressure during the remaining 28 minutes at a flow rate of 13, a total eventual pressure release of 28 * 13 = 364. Then, you could spend your third minute moving to valve CC and your fourth minute opening it, providing an additional 26 minutes of eventual pressure release at a flow rate of 2, or 52 total pressure released by valve CC.



Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed. However, you need to release as much pressure as possible, so you'll need to be methodical. Instead, consider this approach:

    == Minute 1 ==
    No valves are open.
    You move to valve DD.

    == Minute 2 ==
    No valves are open.
    You open valve DD.

    == Minute 3 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve CC.

    == Minute 4 ==
    Valve DD is open, releasing 20 pressure.
    You move to valve BB.

    == Minute 5 ==
    Valve DD is open, releasing 20 pressure.
    You open valve BB.

    == Minute 6 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve AA.

    == Minute 7 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve II.

    == Minute 8 ==
    Valves BB and DD are open, releasing 33 pressure.
    You move to valve JJ.

    == Minute 9 ==
    Valves BB and DD are open, releasing 33 pressure.
    You open valve JJ.

    == Minute 10 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve II.

    == Minute 11 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve AA.

    == Minute 12 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve DD.

    == Minute 13 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve EE.

    == Minute 14 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve FF.

    == Minute 15 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve GG.

    == Minute 16 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You move to valve HH.

    == Minute 17 ==
    Valves BB, DD, and JJ are open, releasing 54 pressure.
    You open valve HH.

    == Minute 18 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve GG.

    == Minute 19 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve FF.

    == Minute 20 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You move to valve EE.

    == Minute 21 ==
    Valves BB, DD, HH, and JJ are open, releasing 76 pressure.
    You open valve EE.

    == Minute 22 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve DD.

    == Minute 23 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You move to valve CC.

    == Minute 24 ==
    Valves BB, DD, EE, HH, and JJ are open, releasing 79 pressure.
    You open valve CC.

    == Minute 25 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 26 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 27 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 28 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 29 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

    == Minute 30 ==
    Valves BB, CC, DD, EE, HH, and JJ are open, releasing 81 pressure.

This approach lets you release the most pressure possible in 30 minutes with this valve layout, 1651.

Work out the steps to release the most pressure in 30 minutes. What is the most pressure you can release?